# Tweet Preprocessing

In [1]:
# DS essentials
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# python support
import re
import string
import datetime
import pickle 
from collections import Counter

# visualizations
from wordcloud import WordCloud
from nltk import FreqDist

# NLTK
import nltk
from nltk.corpus import words, stopwords, wordnet

# spacy
import spacy

from preprocessing_funcs import clean_tweet, get_hashtags

In [2]:
text = pd.read_csv('../data_files/tweets.csv.zip')

In [3]:
text.head()

,user_id,user_key,created_at,created_str,retweet_count,retweeted,favorite_count,text,tweet_id,source,hashtags,expanded_urls,posted,mentions,retweeted_status_id,in_reply_to_status_id
0,1.868981e+09,ryanmaxwell_1,1.458672e+12,2016-03-22 18:31:42,NaN,NaN,NaN,#IslamKills Are you trying to say that there w...,7.123460e+17,NaN,"[""IslamKills""]",[],POSTED,[],NaN,NaN
1,2.571870e+09,detroitdailynew,1.476133e+12,2016-10-10 20:57:00,0.0,False,0.0,"Clinton: Trump should’ve apologized more, atta...",7.855849e+17,"<a href=""http://twitterfeed.com"" rel=""nofollow...",[],"[""http://detne.ws/2e172jF""]",POSTED,[],NaN,NaN
2,1.710805e+09,cookncooks,1.487767e+12,2017-02-22 12:43:43,NaN,NaN,NaN,RT @ltapoll: Who was/is the best president of ...,8.343832e+17,NaN,[],[],POSTED,[],NaN,NaN
3,2.584153e+09,queenofthewo,1.482765e+12,2016-12-26 15:06:41,NaN,NaN,NaN,RT @jww372: I don't have to guess your religio...,8.134006e+17,NaN,"[""ChristmasAftermath""]",[],POSTED,[],NaN,NaN
4,1.768260e+09,mrclydepratt,1.501987e+12,2017-08-06 02:36:24,NaN,NaN,NaN,RT @Shareblue: Pence and his lawyers decided w...,8.940243e+17,NaN,[],[],POSTED,[],NaN,NaN


In [4]:
tweets = text[['text']].astype(str)

In [5]:
tweets.head()

,text
0,#IslamKills Are you trying to say that there w...
1,"Clinton: Trump should’ve apologized more, atta..."
2,RT @ltapoll: Who was/is the best president of ...
3,RT @jww372: I don't have to guess your religio...
4,RT @Shareblue: Pence and his lawyers decided w...


## Tweet cleaning

In [6]:
%%time
tweets['clean'] = tweets['text'].map(clean_tweet)

CPU times: user 6.16 s, sys: 51.1 ms, total: 6.21 s
Wall time: 6.25 s


In [7]:
tweets.head()

,text,clean
0,#IslamKills Are you trying to say that there w...,islamkills are you trying to say that there we...
1,"Clinton: Trump should’ve apologized more, atta...",clinton trump should ve apologized more attack...
2,RT @ltapoll: Who was/is the best president of ...,who was is the best president of the past year...
3,RT @jww372: I don't have to guess your religio...,i don t have to guess your religion christmasa...
4,RT @Shareblue: Pence and his lawyers decided w...,pence and his lawyers decided which of his off...


## Hashtag segmenting and acronym parsing

These are hashtags and acronyms that keep popping up as being very frequently used and/or in topic modeling. Creating a big dictionary to segment or parse these as part of the cleaning process.

In [9]:
hashtag_dict = {'realdonaldtrump': 'real donald trump',
                'hillaryclinton': 'hillary clinton', 
                'wakeupamerica': 'wake up america', 
                'neverhillary': 'never hillary',
                'trumpforpresident': 'trump for president', 
                'islamkills': 'islam kills',
                'blacklivesmatter': 'black lives matter',
                'survivalguidetothanksgiving': 'survival guide to thanksgiving',
                'christmasaftermath': 'christmas aftermath',
                'teaparty': 'tea party',
                'betteralternativetodebates': 'better alternative to debates',
                'crookedhillary': 'crooked hillary',
                'thingsmoretrustedthanhillary': 'things more trusted than hillary',
                'ruinadinnerinonephrase': 'ruin a dinner in one phrase',
                'makemehateyouinonephrase': 'make me hate you in one phrase',
                'idrunforpresidentif': 'id run for president if',
                'ihavearighttoknow': 'i have a right to know',
                'hrc': 'hillary clinton',
                'foxnews': 'fox news', 
                'stopislam': 'stop islam',
                'icelebratetrumpwith': 'i celebrate trump with',
                'reasonstoprotest': 'reasons to protest',
                'trumpsfavoriteheadline': 'trumps favorite headline',
                'merkelmussbleiben': 'merkel muss bleiben',
                'rejecteddebatetopics': 'rejected debate topics',
                'makeamericagreatagain': 'make america great again',
                'todolistbeforechristmas': 'to',
                'debatenight': 'debate night',
                'cruzcrew': 'cruz crew',
                'tedcruz': 'ted cruz',
                'draintheswamp': 'drain the swamp',
                'potuslasttweet': 'potus last tweet',
                'giftideasforpoliticians': 'gift ideas for politicians', 
                'hillaryshealth': 'hillarys health',
                'hillaryforprison': 'hillary for prison',
                'thingspeopleontwitterlike': 'things people on twitter like',
                'obamaswishlist': 'obamas wish list',
                'trumpbecause': 'trump because',
                'nocybercensorship': 'no cyber censorship',
                'votetrump': 'vote trump',
                'nevertrump': 'never trump',
                'americafirst': 'america first',
                'demdebate': 'dem debate',
                'gopdebate': 'gop debate',
                'rednationrising': 'red nation rising',
                'hillarysemails': 'hillarys emails',
                'clintonoriginalbirther': 'clinton orginal birther',
                'trumptrain': 'trump train',
                'oscarhasnocolor': 'oscar has no color', 
                'clintonoriginalbirther': 'clinton original birther',
                'basketofdeplorables': 'basket of deplorables',
                'billclinton': 'bill clinton',
                'imwithher': 'im with her',
                'nowplaying': 'now playing',
                'opiceisis': 'op ice isis',
                'iceisis': 'ice isis', 
                'tcot': 'top conservatives on twitter',
                'ccot': 'christian conservatives on twitter',
                'pjnet': 'patriot journalist network'
               }

In [10]:
%%time
tweets['clean'] = tweets['clean'].replace(hashtag_dict, regex=True)

CPU times: user 10.3 s, sys: 55.6 ms, total: 10.3 s
Wall time: 10.4 s


In [11]:
tweets.head()

,text,clean
0,#IslamKills Are you trying to say that there w...,islam kills are you trying to say that there w...
1,"Clinton: Trump should’ve apologized more, atta...",clinton trump should ve apologized more attack...
2,RT @ltapoll: Who was/is the best president of ...,who was is the best president of the past year...
3,RT @jww372: I don't have to guess your religio...,i don t have to guess your religion christmas ...
4,RT @Shareblue: Pence and his lawyers decided w...,pence and his lawyers decided which of his off...


## Remove unwanted words

- lots of german
- stop words
- other weird words

### Uniquely German Words

In [12]:
# a list of all english words known to nltk
english_words = list(nltk.corpus.words.words())
english_words = [word.lower() for word in english_words]
len(english_words), type(english_words)

(236736, list)

In [13]:
word_net = list(nltk.corpus.wordnet.words())
word_net = [word.lower() for word in word_net]
len(word_net), type(word_net)

(147306, list)

In [14]:
many_english_words = set(english_words + word_net)
len(many_english_words), type(many_english_words)

(323592, set)

In [15]:
from nltk.corpus.europarl_raw import german

german = list(german.words())
german = [word.lower() for word in german]
len(german), type(german)

(556226, list)

In [16]:
german_stop_words = stopwords.words("german")
len(german_stop_words), type(german_stop_words)

(232, list)

In [17]:
german_words = set(german + german_stop_words)
len(german_words)

29839

In [18]:
unique_german = set([word.lower() for word in german if word not in many_english_words])
len(unique_german)

28154

### Stop Words

In [19]:
standard_stop_words = stopwords.words("english")
print(standard_stop_words)
len(standard_stop_words), type(standard_stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

(179, list)

In [21]:
# Some twitter stopwords

with open('../data_files/twitter_stopwords.txt') as f:
    words = f.read().split(',')
    twitter_stopwords = list(words)
    twitter_stopwords = [word.lower() for word in twitter_stopwords]

len(twitter_stopwords)

624

In [36]:
# other things that have popped up in preprocessing
other_stops = ['amp', '…', 'll', '\ufeff1']

In [37]:
english_stops = set(standard_stop_words + twitter_stopwords + other_stops)

### Big word removal set

In [38]:
words_to_remove = set(list(unique_german) + list(english_stops))

In [39]:
len(words_to_remove), type(words_to_remove)

(28768, set)

## spaCy Pipeline

In [26]:
# keep ner, entity_linker
disabled_components = ['parser', 'tagger']

nlp = spacy.load('en_core_web_sm', disable=disabled_components)

In [27]:
nlp.add_pipe(nlp.create_pipe('sentencizer'))

In [28]:
print(nlp.pipe_names)

['ner', 'sentencizer']


In [29]:
# as much as it pains me...
def get_me_mike_pence(text):
    return text.replace("penny", "pence")

In [30]:
def lemmatize_pipe(doc):
    lemma_list = [str(tok.lemma_) for tok in doc
                 if tok.is_alpha and tok.text not in words_to_remove]
    lem_string = " ".join(lemma_list)
    # get back mike pence
    lem_string = get_me_mike_pence(lem_string)
    return lem_string

def preprocess_pipe(texts, batch_size=100):
    preproc_pipe = []
    for doc in nlp.pipe(texts, batch_size=batch_size):
        preproc_pipe.append(lemmatize_pipe(doc))
    return preproc_pipe

In [40]:
%%time
tweets['processed'] = preprocess_pipe(tweets['clean'], batch_size=10000)

CPU times: user 1min 37s, sys: 21.5 s, total: 1min 59s
Wall time: 1min 59s


In [41]:
tweets.head()

,text,clean,processed
0,#IslamKills Are you trying to say that there w...,islam kills are you trying to say that there w...,islam kill try say terrorist attack europe ref...
1,"Clinton: Trump should’ve apologized more, atta...",clinton trump should ve apologized more attack...,clinton trump apologize attack little
2,RT @ltapoll: Who was/is the best president of ...,who was is the best president of the past year...,well president past retweet
3,RT @jww372: I don't have to guess your religio...,i don t have to guess your religion christmas ...,guess religion christmas aftermath
4,RT @Shareblue: Pence and his lawyers decided w...,pence and his lawyers decided which of his off...,pence lawyer decide official email public can see


## Word Frequencies

Finding most frequent words that might need cleaning still, especially hashtags and acronyms that might need to be added to the dictionary above.

In [42]:
# top words in processed that are NOT in many_english_words
def word_freq(text, num_words=10, freq_thresh=1000):
    all_words = []
    for tweet in list(text):
        words = tweet.split()
        for word in words:
            if word not in many_english_words:
                all_words.append(word.lower())
    
    top_words = Counter(all_words).most_common(num_words)
    
    results_list = []
    for pair in top_words:
        if pair[1] > freq_thresh:
            results_list.append(pair)
            
    return results_list

In [43]:
word_freq(tweets['clean'], num_words=10, freq_thresh=500)

[('obama', 9788),
 ('has', 6150),
 ('conservatives', 5824),
 ('says', 3105),
 ('potus', 2120),
 ('merkel', 2092),
 ('women', 2058),
 ('rt', 2051),
 ('ll', 1913),
 ('cruz', 1910)]

In [44]:
word_freq(tweets['processed'], num_words=10, freq_thresh=500)

[('obama', 9788),
 ('\ufeff1', 2388),
 ('potus', 2120),
 ('merkel', 2092),
 ('cruz', 1910),
 ('gt', 1657),
 ('cnn', 1547),
 ('americans', 1517),
 ('dnc', 1453),
 ('htt', 1289)]

## Pickle clean tweets for later use!

In [45]:
tweets.head(3)

,text,clean,processed
0,#IslamKills Are you trying to say that there w...,islam kills are you trying to say that there w...,islam kill try say terrorist attack europe ref...
1,"Clinton: Trump should’ve apologized more, atta...",clinton trump should ve apologized more attack...,clinton trump apologize attack little
2,RT @ltapoll: Who was/is the best president of ...,who was is the best president of the past year...,well president past retweet


In [46]:
tweets = tweets[['text', 'processed']]

In [47]:
with open('../data_files/processed_tweets.pickle', 'wb') as to_write:
   pickle.dump(tweets, to_write)